# Двумерная инерциальная навигационная система на основе данных смартфона

## Введение

В этой лабораторной работе мы реализуем алгоритм простой двумерной инерциальной навигационной системы (ИНС) на основе данных акселерометра и гироскопа смартфона.

## Теоретическая часть

Для получения ускорений в стартовой системе координат необходимо учитывать ориентацию устройства. Поэтому целиком алгоритм двумерной инерциальной навигационной системы состоит из 4х основных шагов:

1. **Расчет угла поворота вокруг вертикальной оси**
2. **Преобразование ускорений**
3. **Интегрирование ускорения для получения скорости**
4. **Интегрирование скорости для получения положения**

## Практическая реализация

### Загрузка данных
  
В вашем мобильном телефоне (сейчас уже практически в любом) есть набор датчиков: блок акселерометров, блок гироскопов и магнитометр. Поэтому в рамках лабораторной работы мы будем пользоваться вашим телефоном.  
Для записей воспользуемся приложением **Sensor Logger**:  
https://play.google.com/store/apps/details?id=com.kelvin.sensorapp&hl=en&gl=US  
Для Apple можноно воспользоваться вот этим приложением или аналогичным (работа рассчитана на приложение *Sensor Logger*, однако сходные приложения так же подходят, в случае возникновения сложностей обратитесь к преподавателю):  
https://apps.apple.com/us/app/physics-toolbox-sensor-suite/id1128914250  
Функция для чтения данных из файлов записи приложения **Sensor Logger** находится в следующей ячейке, можете использовать ее. 

In [5]:
from matplotlib import pyplot as plt
import csv


def read_csv(fname):
    time = []
    x = []
    y = []
    z = []
    init = False
    with open(fname, newline = '') as csvfile:
        datareader = csv.reader(csvfile)
        for row in datareader:
            if (len(row)>=4):
                if init:
                    time.append((float(row[1])))
                    z.append(float(row[2]))
                    y.append(float(row[3]))
                    x.append(float(row[4]))
                else:
                    init=True
    return time, x, y, z

# t_w, w_x, w_y, w_z = read_csv('Gyroscope.csv')      # Пример чтения файла записи гироскопа
# t_a, a_x, a_y, a_z = read_csv('Accelerometer.csv')  # Пример чтения файла записи акселерометра
# plt.plot(t_w,w_x,t_w,w_y,t_w,w_z)                   # Построение графиков

### Подготовка
Прежде чем использовать данные необходимо понять, что они нам дают. В случае акселерометров и гироскопов, необходимо понять направления осей чувствительности этих датчиков. Так же необходимо проверить подвергаются ли данные какой либо обработке.  
Используя приложение определите направления осей чувствительности датчиков, так же опишите своими словами что хранится в каждом из файлов записи приложения **Sensor Logger**.  
В следующей ячейке можете написать код в случае необходимости, если код не нужен удалите эту ячейку.

Замените это изображение на изображение вашего телефона с осями чувствительности ваших сенсоров. Наименование осей должно совпадать с наименованием осей в примере чтения данных в выданной функции.  
<img src="Smartphone.png" alt="Smartphone" width="300" height="300">

Опишите тип данных хранящихся в файлах записи приложения **Sensor Logger** при записи данных акселерометра и гироскопа:  
**Accelerometer.csv** -  
**Gyroscope.csv** -  
**AccelerometerUncalibrated.csv** -  
**GyroscopeUncalibrated.csv** -  
**TotalAcceleration.csv** -  

### Вычисление угла поворота

Интегрируем угловую скорость для получения угла поворота на каждый момент времени. Списки угловой скорости и линейного ускорения могут иметь разную длину, поэтому при реализации интегрирования обзяательно используйте разные имена для списка времени измерения угловой скорости и списков времени измерения линейного ускорения. Интегрирование угловой скорости выполняется методом прямоугольников согласно формуле:
\begin{align}
\theta(t) = \theta_0 + \int_{t_0}^{t} \omega_z(\tau) \, d\tau
\end{align}

\begin{align}
\theta(t) = \theta(t-1) + \omega_z(t)*dt
\end{align}

Где:
- $omega_z$ — угловая скорость вокруг оси Z.

Реализуйте алгоритм в следующей ячейке.

### Преобразование ускорений в стартовую систему координат

Дле перевода ускорений в стартовую систему координат воспользуемся матрицей направляющих косинусов:  
\begin{align}
    \begin{pmatrix}
        a_x^{\text{старт}} \\
        a_y^{\text{старт}}
    \end{pmatrix}
    =
    \begin{pmatrix}
        \cos\theta & -\sin\theta \\
        \sin\theta & \cos\theta
    \end{pmatrix}
    \begin{pmatrix}
        a_x^{\text{св}} \\
        a_y^{\text{св}}
    \end{pmatrix}
\end{align}

Для синхронизации акселерометра и гироскопа можно воспользоваться методом интерполяции из библиотеки numpy. Пример интерполяции приведен в следующей ячейке. Выполните перевод ускорений из связанной системы координат в стартовую в следующей ячейке.

In [48]:
import numpy as np

# theta_interp = np.interp(t_a, t_w, theta)

### Интегрирование ускорения для получения скорости
Интегрирование ускорений выполняется аналогично интегрированию угловых скоростей методом прямоугольников:
\begin{align}
    v(t) = v_0 + \int_{t_0}^{t} a(\tau) \, d\tau
\end{align}

\begin{align}
    v(t) = v(t-1) + a(t)*dt
\end{align}

,где  
$v$ - линейная скорость вдоль одной из осей  
$a$ - ускорение вдоль соответствующей оси

Выполните соответствующие операции в следующей ячейке.

### Интегрирование скорости для получения положения
Аналогичным образом интегрируется и скорость:
\begin{align}
    s(t) = s_0 + \int_{t_0}^{t} v(\tau) \, d\tau
\end{align}

\begin{align}
    s(t) = s(t-1) + s(t)*dt
\end{align}

,где  
$s$ - перемещение вдоль одной из осей  
$v$ - скорость вдоль соответствующей оси

Выполните соответствующие операции в следующей ячейке.

### Визуализация траектории
Постройте график получившейся траектории

## Выводы
Проанализируйте ваши результаты.  
На сколько точной получилась восстановленная траектория?  
В чем причиная такого результата?  
Какой из датчиков имеет большую погрешность?  
Что можно сделать для улучшения результата?  
Ответьте на поставленные вопросы в следующей ячейке.